In [1]:
pip install --upgrade google-cloud-bigquery

     |████████████████████████████████| 174kB 2.8MB/s 
     |████████████████████████████████| 81kB 7.8MB/s 
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.11.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement six~=1.12.0, but you'll have six 1.14.0 which is incompatible.
ERROR: google-cloud-storage 1.16.2 has requirement google-resumable-media<0.5.0dev,>=0.3.1, but you'll have google-resumable-media 0.5.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: google-cloud-core 1.0.3
    Uninstalling google-cloud-core-1.0.3:
      Successfully uninstalled google-cloud-core-1.0.3
  Found existing installation: six 1.12.0
    Uninstalling six-1.12.0:
      Successfully uninstalled six-1.12.0
  Found exist

In [3]:
from google.colab import auth
auth.authenticate_user()

from google.cloud import bigquery

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = 'capstone-247602'
client = bigquery.Client(project=project_id)

for dataset in client.list_datasets():
  print(dataset.dataset_id)

import pandas as pd

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = 'capstone-247602'
df_full = pd.io.gbq.read_gbq('''
SELECT * FROM `capstone-247602.Separate_Aggrigated_Datasets.tbl_Digital_series_event_date`''', 
project_id=project_id, verbose=False)

AuthorizationError: ignored

In [0]:
# Filter main dataframe by SHOW
# Add in calculated features
# Make airdate the index
# create dataframe of features for Average Mins Viewed and Imps

import warnings
warnings.filterwarnings("ignore")

show_name = "SUPERSTORE"
show_table = df_full[df_full.Show_Name == show_name]

show_table['days_since_last_episode'] = (show_table.groupby('Season')['airdate'].diff().fillna(0).astype(str).str[:2]).astype(str).astype(int)
show_table['days_since_1st_episode'] = ((show_table.airdate - show_table.airdate.min()).astype(str).str[:2]).astype(str).astype(int)

show_table['airdate']= pd.to_datetime(show_table['airdate']) 
show_table = show_table.set_index('airdate')


df_amv = show_table[[ 'average_mins_viewed','days_since_last_episode','days_since_1st_episode','dummy_weekday','dummy_10pm','dummy_9pm','dummy_8pm',
                     'quarter','Median_Age','Median_Income','season_number','Season','episode_number']]

df_I = show_table[[ 'Imps','days_since_last_episode','days_since_1st_episode','dummy_weekday','dummy_10pm','dummy_9pm','dummy_8pm',
                 'quarter','Median_Age','Median_Income','season_number','Season','episode_number']]

forecast_cols = ['average_mins_viewed', 'Imps']
dfs = [df_amv,df_I]

print(len(df_amv),len(df_I))
df_amv.head()

In [0]:
!pip install fbprophet

In [0]:
def fit_predict_model(dataframe, interval_width = 0.99, changepoint_range = 0.8):
    m = Prophet(daily_seasonality = False, yearly_seasonality = False, weekly_seasonality = False,
                seasonality_mode = 'multiplicative', 
                interval_width = interval_width,
                changepoint_range = changepoint_range)
    m = m.fit(dataframe)
    
    forecast = m.predict(dataframe)
    forecast['fact'] = dataframe['y'].reset_index(drop = True)
    print('Displaying Prophet plot')
    fig1 = m.plot(forecast)
    return forecast
    
pred = fit_predict_model(clean_df)

In [0]:
def detect_anomalies(forecast):
    forecasted = forecast[['ds','trend', 'yhat', 'yhat_lower', 'yhat_upper', 'fact']].copy()
    #forecast['fact'] = df['y']

    forecasted['anomaly'] = 0
    forecasted.loc[forecasted['fact'] > forecasted['yhat_upper'], 'anomaly'] = 1
    forecasted.loc[forecasted['fact'] < forecasted['yhat_lower'], 'anomaly'] = -1

    #anomaly importances
    forecasted['importance'] = 0
    forecasted.loc[forecasted['anomaly'] ==1, 'importance'] = \
        (forecasted['fact'] - forecasted['yhat_upper'])/forecast['fact']
    forecasted.loc[forecasted['anomaly'] ==-1, 'importance'] = \
        (forecasted['yhat_lower'] - forecasted['fact'])/forecast['fact']
    
    return forecasted

pred = detect_anomalies(pred)

In [0]:
def plot_anomalies(forecasted):
    interval = alt.Chart(forecasted).mark_area(interpolate="basis", color = '#7FC97F').encode(
    x=alt.X('ds:T',  title ='date'),
    y='yhat_upper',
    y2='yhat_lower',
    tooltip=['yearmonthdate(ds)', 'fact', 'yhat_lower', 'yhat_upper']
    ).interactive().properties(
        title='Anomaly Detection'
    )

    fact = alt.Chart(forecasted[forecasted.anomaly==0]).mark_circle(size=15, opacity=0.7, color = 'Black').encode(
        x='ds:T',
        y=alt.Y('fact', title='Sunspots'),    
        tooltip=['yearmonthdate(ds)', 'fact', 'yhat_lower', 'yhat_upper']
    ).interactive()

    anomalies = alt.Chart(forecasted[forecasted.anomaly!=0]).mark_circle(size=30, color = 'Red').encode(
        x='ds:T',
        y=alt.Y('fact', title='Sunspots'),    
        tooltip=['yearmonthdate(ds)', 'fact', 'yhat_lower', 'yhat_upper'],
        size = alt.Size( 'importance', legend=None)
    ).interactive()

    return render(alt.layer(interval, fact, anomalies)\
              .properties(width=870, height=450)\
              .configure_title(fontSize=20))
              
plot_anomalies(pred)